# IMDB Review Sentiment Analysis

## Dataset and Exploration

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('./data/IMDB_Dataset.csv')

In [6]:
df.shape

(50000, 2)

In [7]:
df.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [8]:
df.loc[df['sentiment'] == 'positive', 'sentiment'] = 1
df.loc[df['sentiment'] == 'negative', 'sentiment'] = 0

In [9]:
df.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


## Preprocessing

In [10]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /Users/arths/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/arths/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/arths/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [11]:
def clean_strings(df):
    def remove_tags(text):
        tag_pattern = re.compile(r'<[^>]+>')
        return tag_pattern.sub('', text)
    
    def remove_stop_words(text):
        stop_pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
        return stop_pattern.sub('', text)

    corpus = []
    text_data = ''
    for i in tqdm(range(len(df.index)), ascii=False, ncols=100):
        
        # remove html tags (since this is webscraped)
        text_data = remove_tags(df['review'][i])

        # remove any non-text characters
        text_data = re.sub('[^a-zA-Z]', ' ', text_data)

        # lowercase
        text_data = text_data.lower()

        # remove stopwords
        text_data = remove_stop_words(text_data)
        
        # lower case and make array
        text_data = text_data.split()

        # lemmatize each word
        wl = WordNetLemmatizer()
        text_data = [wl.lemmatize(word) for word in text_data]

        # append to corpus
        corpus.append(text_data)

    return corpus

In [12]:
cleaned_sentences = clean_strings(df)

100%|███████████████████████████████████████████████████████| 50000/50000 [00:33<00:00, 1509.03it/s]


In [13]:
print(df['review'][0])
print(cleaned_sentences[0])

One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fac

## Transfer Learning w/ GLoVe

In [14]:
from gensim.test.utils import get_tmpfile, datapath
from gensim.models import KeyedVectors, Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec
import os
import numpy as np

In [ ]:
# download 6 billion glove dataset from https://nlp.stanford.edu/data/glove.6B.zip

In [15]:
# get glove file and make temp glove word2vec file
glove_file = datapath(os.path.abspath('./glove.6B.100d.txt'))
glove2vec_file = get_tmpfile(os.path.abspath('./glove_word2vec_6B.100d.txt'))

# convert glove embeddings to word2vec format
_ = glove2word2vec(glove_file, glove2vec_file)
glove_vectors = KeyedVectors.load_word2vec_format(glove2vec_file)

/var/folders/bm/v9sftty97f1b1pjtvxw68byc0000gn/T/ipykernel_3488/1871904394.py:6: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  _ = glove2word2vec(glove_file, glove2vec_file)


In [112]:
# build initial embedding on our dataset
base_model = Word2Vec(vector_size=100, min_count=5)
base_model.build_vocab(cleaned_sentences)

# train on our dataset
base_model.train(cleaned_sentences, total_examples=base_model.corpus_count, epochs=base_model.epochs)
base_model_wv = base_model.wv

del base_model

In [113]:
# build initial embedding on our dataset
glove_model = Word2Vec(vector_size=100, min_count=5)
glove_model.build_vocab(cleaned_sentences)
total_examples = glove_model.corpus_count

# add GLoVe vocab and weights
glove_model.build_vocab(glove_vectors.index_to_key, update=True)

# # train on dataset
glove_model.train(cleaned_sentences, total_examples=total_examples, epochs=glove_model.epochs)
glove_model_wv = glove_model.wv

del glove_model

In [114]:
base_model_wv.most_similar('romance')

[('romantic', 0.822129487991333),
 ('triangle', 0.6487324833869934),
 ('intrigue', 0.599744975566864),
 ('drama', 0.5786858797073364),
 ('friendship', 0.5746517181396484),
 ('screwball', 0.5686193108558655),
 ('bittersweet', 0.5652892589569092),
 ('sentimental', 0.5648941397666931),
 ('tale', 0.5623289942741394),
 ('adventure', 0.5554719567298889)]

In [115]:
glove_model_wv.most_similar('romance')

[('romantic', 0.8137059211730957),
 ('triangle', 0.6177476644515991),
 ('fairytale', 0.6133526563644409),
 ('drama', 0.6042848229408264),
 ('intrigue', 0.6025968790054321),
 ('screwball', 0.5895823240280151),
 ('bittersweet', 0.5888437032699585),
 ('sappy', 0.5846787095069885),
 ('sentimental', 0.5700287818908691),
 ('sweet', 0.5632114410400391)]

## Preparing Data

In [116]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [117]:
def prep_data(embedding_model_wv, random_state=19):

    # cleaned strings
    X_raw = cleaned_sentences
    # 0 for negative, 1 for positive
    y_raw = df['sentiment']


    # split randomly
    X_train, X_test, y_train, y_test = train_test_split(X_raw, y_raw, test_size=0.2, random_state=19)


    # tokenize using embedding model
    word_tokenizer = Tokenizer()
    word_tokenizer.word_index = embedding_model_wv.key_to_index

    X_train = word_tokenizer.texts_to_sequences(X_train)
    X_test = word_tokenizer.texts_to_sequences(X_test)

    maxlen = 0
    for X in X_train:
        maxlen = max(maxlen, len(X))

    # pad all sequences and convert to numpy arrays
    X_train = pad_sequences(X_train, padding='post', maxlen=maxlen, value=len(embedding_model_wv.key_to_index))
    X_test = pad_sequences(X_test, padding='post', maxlen=maxlen, value=len(embedding_model_wv.key_to_index))

    X_train, y_train = np.array(X_train, dtype=np.int64), np.array(y_train, dtype=np.int64)
    X_test, y_test = np.array(X_test, dtype=np.int64), np.array(y_test, dtype=np.int64)

    
    # create embedding matrix with extra row for padding
    glove_embedding = embedding_model_wv.vectors
    pad_embedding = np.zeros((1, glove_embedding.shape[1]))

    embedding_matrix = np.vstack((glove_embedding, pad_embedding))


    return X_train, X_test, y_train, y_test, embedding_matrix

In [118]:
base_X_train, base_X_test, base_y_train, base_y_test, base_embedding = prep_data(base_model_wv)
glove_X_train, glove_X_test, glove_y_train, glove_y_test, glove_embedding = prep_data(glove_model_wv)

## Build and Train Models

In [119]:
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, GlobalMaxPooling1D, Embedding, Conv1D, LSTM

In [132]:
def train_and_test_model(model, X_train, X_test, y_train, y_test):

    print('Training Model...')
    model_hist = model.fit(X_train, y_train, batch_size=64, epochs=5, verbose=1, validation_split=0.2)

    print('Testing Model...')
    score = model.evaluate(X_test, y_test, verbose=1)

    return model_hist, score

### CNN

In [120]:
def build_cnn(embedding, X_train):
    cnn = Sequential(
        layers=(
            # use our GLoVe-based transfer-learned embedding
            Embedding(*embedding.shape, weights=[embedding], input_length=X_train.shape[1], trainable=False),
            # convolution to 128-dim space
            Conv1D(128, 5, activation='relu'),
            GlobalMaxPooling1D(),
            Dense(1, activation='sigmoid')
        )
    )

    cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

    print(cnn.summary())

    return cnn

In [122]:
base_cnn = build_cnn(base_embedding, base_X_train)

base_cnn_hist, base_cnn_score = train_and_test_model(base_cnn, base_X_train, base_X_test, base_y_train, base_y_test)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 1362, 100)         3474600   
                                                                 
 conv1d_5 (Conv1D)           (None, 1358, 128)         64128     
                                                                 
 global_max_pooling1d_5 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 3,538,857
Trainable params: 64,257
Non-trainable params: 3,474,600
_________________________________________________________________
None
Training CNN...
Epoch 1/5
500/500 [==============================] - 80s 159ms/step - loss: 0.3707 -

In [123]:
glove_cnn = build_cnn(glove_embedding, glove_X_train)

glove_cnn_hist, glove_cnn_score = train_and_test_model(glove_cnn, glove_X_train, glove_X_test, glove_y_train, glove_y_test)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 1362, 100)         3491400   
                                                                 
 conv1d_6 (Conv1D)           (None, 1358, 128)         64128     
                                                                 
 global_max_pooling1d_6 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_6 (Dense)             (None, 1)                 129       
                                                                 
Total params: 3,555,657
Trainable params: 64,257
Non-trainable params: 3,491,400
_________________________________________________________________
None
Training CNN...
Epoch 1/5
500/500 [==============================] - 82s 164ms/step - loss: 0.3655 -

## LSTM

In [130]:
def build_lstm(embedding, X_train):
    lstm = Sequential(
        layers=(
            # use our GLoVe-based transfer-learned embedding
            Embedding(*embedding.shape, weights=[embedding], input_length=X_train.shape[1], trainable=False),
            # convolution to 128-dim space
            LSTM(128),
            Dense(1, activation='sigmoid')
        )
    )

    lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

    print(lstm.summary())

    return lstm

In [ ]:
base_lstm = build_lstm(base_embedding, base_X_train)

base_lstm_hist, base_lstm_score = train_and_test_model(base_lstm, base_X_train, base_X_test, base_y_train, base_y_test)

In [ ]:
glove_lstm = build_lstm(glove_embedding, glove_X_train)

glove_lstm_hist, glove_lstm_score = train_and_test_model(glove_lstm, glove_X_train, glove_X_test, glove_y_train, glove_y_test)